In [ ]:
from datetime import datetime, timedelta

import fitfile
import pandas as pd
from garmindb import ConfigManager
from garmindb.garmindb import (
    Activities,
    ActivitiesDb,
    Attributes,
    GarminDb,
    garmin_summary_db,
)
from IPython.display import display_markdown

In [ ]:
# Local import
from garmin_analysis.running_activity import RunningActivity

In [ ]:
db_params_dict = ConfigManager.get_db_params()

garmin_db = GarminDb(db_params_dict)
garmin_summary_db = garmin_summary_db.GarminSummaryDb(db_params_dict)
garmin_act_db = ActivitiesDb(db_params_dict)
measurement_system = Attributes.measurements_type(garmin_db)
unit_strings = fitfile.units.unit_strings[measurement_system]
distance_units = {"kilometers": "km"}[
    unit_strings[fitfile.units.UnitTypes.distance_long]
]

In [ ]:
running_activities = Activities.get_by_sport(garmin_act_db, "running")

In [ ]:
last_7_days = [
    x for x in running_activities if x.start_time > datetime.now() - timedelta(days=7)
]

In [ ]:
data = []
for x in running_activities:
    data.append({"start_time": x.start_time, "distance": x.distance})

In [ ]:
df = pd.DataFrame(data).set_index("start_time")

In [ ]:
df2 = df.resample("1d").sum()

In [ ]:
rolling = 14
start_date = datetime(2019, 1, 1)
df2.loc[df2.index >= start_date].rolling(rolling).mean().plot(
    title=f"{rolling}-day rolling mean distance in km"
)